In [12]:
import pandas as pd
from plspider_parser import load
# load the seed and the moves from plspider.txt
# convert moves to my format (0-based index and num_cards instead of index)
data = load('plsmall.txt')

In [18]:
from showdeal import get_form_pysol_seed
# generate deck and tableau from seed
data[['Deck','Tableau']] = data['Seed'].apply(get_form_pysol_seed).apply(pd.Series)

In [15]:
# verify gameplay is valid using my engine
from SpiderSolitar import SpiderSolitaire
from spider_display import tableau_to_string, deck_to_string
deck, tableau, moves = data[['Deck','Tableau','Moves']].values[0]
game = SpiderSolitaire(4,13,2,deck,tableau).copy()
for move in moves:
    # print(tableau_to_string(game), deck_to_string(game), sep='\n')
    # print(move)
    if move == 'deal':
        game.deal()
    elif move[0] == 'clear':
        # dont do anything on clear moves, because in my engine, its auto clearing
        pass
    else:
        source_pile, dest_pile, num_cards = move
        game.move_card(source_pile, dest_pile, num_cards)
    g = game.copy()
    states.append((g.deck,g.tableau))


In [28]:

#%conda install -c conda-forge ipywidgets
from IPython.display import display
import ipywidgets as widgets
import time

deck, tableau, moves = data[['Deck','Tableau','Moves']].values[0]
game = SpiderSolitaire(4,13,2,deck,tableau).copy()

# store all games states
states = []
# add inital state
g = game.copy()
states.append((g.deck,g.tableau))
# add sequencially all other sates
for move in moves:
    if move == 'deal':
        game.deal()
    elif move[0] == 'clear':
        # dont do anything on clear moves, because in my engine, its auto clearing
        pass
    else:
        source_pile, dest_pile, num_cards = move
        game.move_card(source_pile, dest_pile, num_cards)
    g = game.copy()
    states.append((g.deck,g.tableau))

# generate the interactive GUI
def display(move_number):
    if move_number < len(states):
        deck, tableau = states[move_number]
        game = SpiderSolitaire(4,13,2,deck,tableau)
        print(tableau_to_string(game), deck_to_string(game), sep='\n')
        
play = widgets.Play(
    value=0,
    min=0,
    max=len(states),
    step=1,
    interval=500,
    description="Press play",
    disabled=False
)
out = widgets.interactive_output(display, {'move_number': play})

slider = widgets.IntSlider(max=len(states))
widgets.jslink((play, 'value'), (slider, 'value'))
controls = widgets.HBox([play, slider])
widgets.VBox([controls, out])